In [2]:
!pip install tensorflow
!pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.2 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 6.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.6 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 7.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 5.9 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 7.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [207]:
import os
import numpy as np
import random

import gym
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete

from stable_baselines3 import PPO
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [208]:
ENV_NAME = 'TRIA-RL-MODEL3D'

In [209]:
TINI= 50.0; HINI= 20.0; AINI= 1000 

TMIN =-40.0; TMAX=120; HMIN=0.0; HMAX=100.0; AMIN=0.0; AMAX=2000.0

EQUI_LEN= 20

R1= 1; R2= 2; R3= 3; NR3= -3

D = {
     0 : [65.0, 80.0, 50.0, 85.0, 40.0, 90.0], 
     1 : [30.0, 50.0, 20.0, 60.0, 10.0, 70.0], 
     2 : [0.0, 19.0, 200.0, 599.0, 600.0, 2000.0]
    }

In [381]:
class TriaEnv(Env):
    
    def __init__(self):
        self.action_space = MultiDiscrete(np.array([2,2,2,2,2]))
        
        self.observation_space = Box(low=np.array([TMIN, HMIN, AMIN]), high=np.array([TMAX, HMAX, AMAX]))
        
        self.state = [TINI + random.uniform(-2,2), HINI + random.uniform(-2,2), AINI + random.uniform(-2,2)]
        
        print('^^^', self.state, self.state, self.action_space)
        
        self.equilibrium_length = EQUI_LEN
        
    def step(self, action):
        
        actionPrime = [1 if e == 1 else -1 for e in action]
        
        print('***',actionPrime, self.state)
        
        self.state = [a + b for a, b in zip(actionPrime, self.state)]
        
        print('&&&', self.state)
        
        #reduce tria simulation length by 1 second
        self.equilibrium_length -= 1
        
        reward = [R3 if e >= D[i][0] and e<= D[i][1] else R2 if e >= D[i][2] and e<= D[i][3] else R1 if e >= D[i][4] and e < D[i][5] else NR3 for i,e in enumerate(self.state)]
        
        print('$$$', reward)
            
        if self.equilibrium_length <= 0:
            terminated = True
        else:
            terminated = False
            
        info = {}
        
        return self.state, reward, terminated, False, info
    
    def render(self):
        pass
    
    def reset(self):
        
        self.state =[TINI + random.uniform(-2,2),HINI + random.uniform(-2,2),AINI + random.uniform(-2,2)]
        print('@@@', self.state)
        self.equilibrium_length = EQUI_LEN
        
        return self.state, {}
        

In [382]:
env= TriaEnv()

^^^ [48.29658577325382, 18.574057122878266, 998.8055422211961] [48.29658577325382, 18.574057122878266, 998.8055422211961] MultiDiscrete([2 2 2 2 2])


In [383]:
env.observation_space.sample()

array([ 103.7688  ,   28.430658, 1759.992   ], dtype=float32)

In [384]:
env.action_space.sample()

array([0, 0, 0, 1, 1])

In [385]:
env.state

[48.29658577325382, 18.574057122878266, 998.8055422211961]

In [386]:
#env.reset()

In [387]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    #print(state)
    terminated = False
    score = [0,0,0] 
    
    while not terminated:
        #env.render()
        action = env.action_space.sample()
        #print(action, terminated , reward)
        #print(env.step(action))
        next_state, reward, terminated, truncated, info = env.step(action) 
        score = [a + b for a, b in zip(reward, score)]
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

@@@ [50.45522503390718, 20.336204720123703, 1001.6159803036272]
*** [-1, 1, -1, -1, -1] [50.45522503390718, 20.336204720123703, 1001.6159803036272]
&&& [49.45522503390718, 21.336204720123703, 1000.6159803036272]
$$$ [1, 2, 1]
*** [-1, -1, 1, -1, -1] [49.45522503390718, 21.336204720123703, 1000.6159803036272]
&&& [48.45522503390718, 20.336204720123703, 1001.6159803036272]
$$$ [1, 2, 1]
*** [1, 1, -1, 1, -1] [48.45522503390718, 20.336204720123703, 1001.6159803036272]
&&& [49.45522503390718, 21.336204720123703, 1000.6159803036272]
$$$ [1, 2, 1]
*** [1, -1, -1, -1, 1] [49.45522503390718, 21.336204720123703, 1000.6159803036272]
&&& [50.45522503390718, 20.336204720123703, 999.6159803036272]
$$$ [2, 2, 1]
*** [1, 1, 1, -1, 1] [50.45522503390718, 20.336204720123703, 999.6159803036272]
&&& [51.45522503390718, 21.336204720123703, 1000.6159803036272]
$$$ [2, 2, 1]
*** [-1, -1, 1, -1, 1] [51.45522503390718, 21.336204720123703, 1000.6159803036272]
&&& [50.45522503390718, 20.336204720123703, 1001.61

In [388]:
log_path = os.path.join('training','logs')

In [389]:
#from stable_baselines3 import A2C
#from stable_baselines3.common.vec_env import SubprocVecEnv
#env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1) #, tensorboard_log=log_path)
#model = A2C("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [390]:
model.learn(total_timesteps=10)

@@@ [48.33104154984506, 21.789975349864353, 1001.4542799391966]
*** [-1, 1, -1, 1, -1] [48.33104154984506, 21.789975349864353, 1001.4542799391966]
&&& [47.33104154984506, 22.789975349864353, 1000.4542799391966]
$$$ [1, 2, 1]


ValueError: setting an array element with a sequence.